In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")

In [2]:
import McExperiment
from McExperiment import *
import importlib
importlib.reload(McExperiment)

<module 'McExperiment' from 'd:\\OneDriveLenovoD\\OneDrive\\A_Fall23\\FIN535\\HW2023\\FIN535-Final-23F\\Dec20\\FIN535-Final-23F\\McExperiment.py'>

In [3]:
data = pd.read_csv('data.csv')
data['month'] = data['month'].apply(lambda x: datetime.strptime(x, '%Y-%m')) #assume each date per month is the last day
data.set_index(['month'],inplace=True)

In [6]:
# 5Y max Sharpe Strategy -------------------------------------------------------------------------------
# All value ar risk quoted in a return way (not in a loss way)
ratio1 = [0.54, 0.91, 0.59,	0.63, -0.45]
print(ratio1)
SAVE = 40000
CAP = 50000
g = 0.11
MC = 500
path_length = 120 # 10 year
tax_rate = 0.25
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table, Simu_cap_table = Experiments(MC,120,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=tax_rate)
Summary_table,cap_table = GetSummary(Simu_table, Simu_cap_table, path_length)
Summary_table = Summary_table.fillna(0)
Summary_table.round(2)

[0.54, 0.91, 0.59, 0.63, -0.45]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected Annual_sortino,Expected MDD,ret VaR,ret CVaR,cap VaR,cap CVaR
5Y,277786.71,64603.52,277786.71,0.00,1.38,1.89,0.21,0.44,0.44,-336025.46,-344658.40
7Y,473377.85,82419.95,416810.12,0.41,1.37,1.89,0.23,1.46,1.46,-166784.41,-180082.51
10Y,1072469.31,175362.85,0.00,1.00,1.47,2.06,0.23,2.91,2.83,200359.02,196816.48
